<a href="https://colab.research.google.com/github/sbanerjee2020/Traderbot/blob/master/Reinforcement_Learning_for_Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/llSourcell/Reinforcement_Learning_for_Stock_Prediction
  

Cloning into 'Reinforcement_Learning_for_Stock_Prediction'...
remote: Enumerating objects: 54, done.
remote: Total 54 (delta 0), reused 0 (delta 0), pack-reused 54
Unpacking objects: 100% (54/54), done.


In [0]:
!pwd

/content


In [0]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Mar 30 14:00 .
drwxr-xr-x 1 root root 4096 Mar 30 13:49 ..
drwxr-xr-x 1 root root 4096 Mar 27 20:25 .config
drwxr-xr-x 6 root root 4096 Mar 30 14:00 Reinforcement_Learning_for_Stock_Prediction
drwxr-xr-x 1 root root 4096 Mar 27 20:26 sample_data


In [0]:
import os


In [0]:
os.chdir("Reinforcement_Learning_for_Stock_Prediction")

In [0]:
!pwd

/content/Reinforcement_Learning_for_Stock_Prediction


In [0]:
!cp  agent/agent.py  .


In [0]:
!ls -al

total 44
drwxr-xr-x 6 root root 4096 Mar 30 14:01 .
drwxr-xr-x 1 root root 4096 Mar 30 14:00 ..
drwxr-xr-x 2 root root 4096 Mar 30 14:00 agent
-rw-r--r-- 1 root root 1755 Mar 30 14:01 agent.py
drwxr-xr-x 2 root root 4096 Mar 30 14:00 data
-rw-r--r-- 1 root root 1326 Mar 30 14:00 evaluate.py
-rw-r--r-- 1 root root  737 Mar 30 14:00 functions.py
drwxr-xr-x 8 root root 4096 Mar 30 14:00 .git
drwxr-xr-x 2 root root 4096 Mar 30 14:00 images
-rw-r--r-- 1 root root 1620 Mar 30 14:00 README.md
-rw-r--r-- 1 root root 1443 Mar 30 14:00 train.py


In [0]:
!mkdir model

In [0]:
!ls -al

total 48
drwxr-xr-x 7 root root 4096 Mar 30 14:01 .
drwxr-xr-x 1 root root 4096 Mar 30 14:00 ..
drwxr-xr-x 2 root root 4096 Mar 30 14:00 agent
-rw-r--r-- 1 root root 1755 Mar 30 14:01 agent.py
drwxr-xr-x 2 root root 4096 Mar 30 14:00 data
-rw-r--r-- 1 root root 1326 Mar 30 14:00 evaluate.py
-rw-r--r-- 1 root root  737 Mar 30 14:00 functions.py
drwxr-xr-x 8 root root 4096 Mar 30 14:00 .git
drwxr-xr-x 2 root root 4096 Mar 30 14:00 images
drwxr-xr-x 2 root root 4096 Mar 30 14:01 model
-rw-r--r-- 1 root root 1620 Mar 30 14:00 README.md
-rw-r--r-- 1 root root 1443 Mar 30 14:00 train.py


In [0]:
!python train.py ^GSPC 10 10

Using TensorFlow backend.
Traceback (most recent call last):
  File "train.py", line 1, in <module>
    from agent.agent import Agent
ImportError: No module named agent


In [0]:
%pycat train.py 

In [0]:
!rm train.py

In [0]:
%%writefile train.py
#from agent.agent import Agent
from agent import *

from functions import *
import sys

if len(sys.argv) != 4:
	print("Usage: python train.py [stock] [window] [episodes]")
	exit()

stock_name, window_size, episode_count = sys.argv[1], int(sys.argv[2]), int(sys.argv[3])

agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			#print("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			#print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("--------------------------------")

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	if e % 10 == 0:
		agent.model.save("model/model_ep" + str(e))


Overwriting train.py


In [0]:
!cat train.py

#from agent.agent import Agent
from agent import *

from functions import *
import sys

if len(sys.argv) != 4:
	print("Usage: python train.py [stock] [window] [episodes]")
	exit()

stock_name, window_size, episode_count = sys.argv[1], int(sys.argv[2]), int(sys.argv[3])

agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			#print("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			#print("Sell: " + formatPrice(data[t]) + " |

In [0]:
!python train.py ^GSPC 10 100

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
Episode 0/100
2019-03-30 17:23:48.758565: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-30 17:23:48.758922: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x558f0d8b4dc0 executing computations on platform Host. Devices:
2019-03-30 17:23:48.758957: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use tf.cast instead.
--------------------------------
Total Profit: -$6522.41
--------------------------------
Episode 1/100
--------------------------------
Total Profit: $269.88
--------------------------------
Episode 2/100
--------------------------------
Total Profit: $37.50
--------------------------------
Episode 3/100
--------------------------------
Total Profit: $261.74
--------------------------------
Episode 4/100
-----------------------------

In [0]:
!ls -al ../Reinforcement_Learning_for_Stock_Prediction/model

total 440
drwxr-xr-x 2 root root  4096 Mar 30 17:18 .
drwxr-xr-x 7 root root  4096 Mar 30 14:03 ..
-rw-r--r-- 1 root root 70768 Mar 30 14:05 model_ep0
-rw-r--r-- 1 root root 70768 Mar 30 14:43 model_ep10
-rw-r--r-- 1 root root 70768 Mar 30 15:22 model_ep20
-rw-r--r-- 1 root root 70768 Mar 30 16:00 model_ep30
-rw-r--r-- 1 root root 70768 Mar 30 16:39 model_ep40
-rw-r--r-- 1 root root 70768 Mar 30 17:18 model_ep50
